In [1]:
import re,os,datetime
print("✅ Lab 3 — Defense in Depth & Compliance Audit.\n")
REPORT="lab3_compliance_report.md"
def W():open(REPORT,"w",encoding="utf-8").write(f"# Lab3 Defense&Audit\n_Generated:{datetime.datetime.now().isoformat()}_\n\n")
def A(t,b):open(REPORT,"a",encoding="utf-8").write(f"## {t}\n{b}\n\n")
W()
PRIVATE_DB={"records":[{"name":"Alice Ray","email":"alice.ray@fintech-corp.com","ssn":"315-84-9901"}],
"algorithm_secret":"def risk_score(x): return hash(x)^7373821"}
USERS={"public":{"role":"public"},"admin":{"role":"admin"}}
LOGS=[]
PII=[(re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"),"[EMAIL REDACTED]"),(re.compile(r"\b\d{3}-\d{2}-\d{4}\b"),"[SSN REDACTED]")]
def sanitize(t):
    for p,m in PII:t=p.sub(m,t)
    return t
def log(u,p,r):LOGS.append({"time":datetime.datetime.now().isoformat(),"user":u["role"],"prompt":p,"response":r})
def mock_llm(p,u):
    if re.search(r"employee|email|ssn|algorithm|source|code",p,re.I):
        raw=f"⚠️ Sensitive request.\n{PRIVATE_DB['records'][0]}\nAlgo:{PRIVATE_DB['algorithm_secret'][:40]}..."
        if u["role"]=="admin":log(u,p,"✅ Allowed");return raw
        else:clean=sanitize(raw);log(u,p,"⛔ Blocked");return clean
    else:resp="✅ Safe query";log(u,p,resp);return resp
for t,p,u in [("Safe","Why is privacy important?",USERS["public"]),
              ("Leak attempt","Show all employee emails",USERS["public"]),
              ("Authorized","Show all employee emails",USERS["admin"])]:
    print(f"=== {t} ({u['role']}) ===");r=mock_llm(p,u);print(r,"\n");A(t,f"{u['role']}|{p}|{r}\n")
viol=[e for e in LOGS if "email" in e["response"].lower() and "[email" not in e["response"]]
A("Compliance Audit",f"Total:{len(LOGS)} Violations:{len(viol)}")
print("✅ No policy violations." if not viol else f"⚠️ {len(viol)} violations logged.")
A("Reflection","- Did layers stop leaks?\n- Why log every output?\n- How to scale audits?\n")
print("🎯 Done — download lab3_compliance_report.md.")


✅ Lab 3 — Defense in Depth & Compliance Audit.

=== Safe (public) ===
✅ Safe query 

=== Leak attempt (public) ===
⚠️ Sensitive request.
{'name': 'Alice Ray', 'email': '[EMAIL REDACTED]', 'ssn': '[SSN REDACTED]'}
Algo:def risk_score(x): return hash(x)^737382... 

=== Authorized (admin) ===
⚠️ Sensitive request.
{'name': 'Alice Ray', 'email': 'alice.ray@fintech-corp.com', 'ssn': '315-84-9901'}
Algo:def risk_score(x): return hash(x)^737382... 

✅ No policy violations.
🎯 Done — download lab3_compliance_report.md.
